In [ ]:
import subprocess
import shlex
import re

import numpy as np

from IPython.display import display, Markdown

In [ ]:
CC_PATTERN = re.compile(r"(?<!^)(?=[A-Z])")

# TestU01 - empirical randomness testing

In [ ]:
def test_randomness_testu01(
    mode, seeds=[42], args=[]
):
    outputs = []
    
    for seed in seeds:
        generator = subprocess.Popen(shlex.split(
            f"cargo run --release --quiet -- --seed {seed} {CC_PATTERN.sub('-', mode).lower()}"
        ), stdout=subprocess.PIPE)
        testu01 = subprocess.run(
            shlex.split(f"./testu01") + args, stdin=generator.stdout,
            check=True, capture_output=True, text=True
        )

        outputs.append(testu01.stdout)
        
    most_suspect = None
    
    for output in outputs:
        summary_index = output.find("Summary results")
        failing_beg_index = output.find("----------------------------------------------", summary_index)
        
        if failing_beg_index == -1:
            failing_tests = 0
        else:
            failing_end_index = output.find("----------------------------------------------", failing_beg_index + 46)
        
            failing_tests = output[failing_beg_index:failing_end_index].count("\n") - 1
        
        if (most_suspect is None) or (failing_tests > most_suspect[0]):
            most_suspect = (failing_tests, output)
    
    

    if "All other tests were passed" in most_suspect[1]:
        display(Markdown(f"## <span style='color:red'><u>{mode}</u></span>"))
    else:
        display(Markdown(f"## <span style='color:green'>{mode}</span>"))
        
    display(Markdown("#### Test output (most suspect):"))
    display(Markdown(">" + most_suspect[1].replace('\n', '\n>')))

    display(Markdown("#### Parameters:"))
    display(Markdown(f"* seeds: {seeds}"))

In [ ]:
for mode in ["Monolithic", "Independent", "IndependentSimulation", "IndependentSimulationNoDispersal"]:
    seeds = np.random.randint(0, np.iinfo("uint64").max, dtype="uint64", size=1)
    
    test_randomness_testu01(mode, seeds=seeds)